In [ ]:
# Lab 12 RNN
import numpy as np
import tensorflow as tf

idx2char = ['h', 'i', 'e', 'l', 'o']
# Teach hello: hihell -> ihello(그 다음 문자를 예측하도록 이렇게 학습)
# x_data = [[0, 1, 0, 2, 3, 3]]  # hihell
y_data = [[1, 0, 2, 3, 3, 4]]  # ihello

num_classes = 5
input_dim = 5  # one-hot size, same as hidden_size to directly predict one-hot
sequence_length = 6  # |ihello| == 6
learning_rate = 0.1

x_one_hot = np.array([[[1, 0, 0, 0, 0],    # h 0
                       [0, 1, 0, 0, 0],    # i 1
                       [1, 0, 0, 0, 0],    # h 0
                       [0, 0, 1, 0, 0],    # e 2
                       [0, 0, 0, 1, 0],    # l 3
                       [0, 0, 0, 1, 0]]],  # l 3
                     dtype=np.float32)

y_one_hot = tf.keras.utils.to_categorical(y_data, num_classes=num_classes)
print(x_one_hot.shape)
print(y_one_hot.shape)

tf.model = tf.keras.Sequential()

# make cell and add it to RNN layer
# input_shape = (1,6,5) => number of sequence (batch), length of sequence, size of input dim
cell = tf.keras.layers.LSTMCell(units=num_classes, input_shape=(sequence_length, input_dim))
tf.model.add(tf.keras.layers.RNN(cell=cell, return_sequences=True))

# single LSTM layer can be used as well instead of creating LSTMCell
# tf.model.add(tf.keras.layers.LSTM(units=num_classes, input_shape=(sequence_length, input_dim), return_sequences=True))

# fully connected layer
tf.model.add(tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(units=num_classes, activation='softmax')))

tf.model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(lr=learning_rate),
                 metrics=['accuracy'])

# train
tf.model.fit(x_one_hot, y_one_hot, epochs=50)
tf.model.summary()

predictions = tf.model.predict(x_one_hot)
for i, prediction in enumerate(predictions):
    print(prediction)
    # print char using argmax, dict
    result_str = [idx2char[c] for c in np.argmax(prediction, axis=1)]
    print("\tPrediction str: ", ''.join(result_str))

In [5]:
# Lab12 RNN
import numpy as np
import tensorflow as tf

idx2char = ['h', 'i', 'e', 'l', 'o']
# hihell -> ihello
x_data = [[0,1,0,2,3,3]]
y_data = [[1,0,2,3,3,4]]

nb_classes = 5
input_dim = 5
sequence_length = 6
learning_rate = 0.1

x_one_hot = tf.keras.utils.to_categorical(x_data, input_dim)
y_one_hot = tf.keras.utils.to_categorical(y_data, nb_classes)

print(x_one_hot.shape)
print(y_one_hot.shape)

# model
tf.model = tf.keras.Sequential()
# RNN layer
cell = tf.keras.layers.LSTMCell(input_shape=(sequence_length, input_dim), units = nb_classes)
tf.model.add(tf.keras.layers.RNN(cell= cell, return_sequences=True)) #return_sequences로 진행중 time step을 알수있음
# fully connected layer
tf.model.add(tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(units=nb_classes, activation='softmax')))
tf.model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), metrics=['accuracy'])
#train
tf.model.fit(x_one_hot,y_one_hot,epochs=50)
tf.model.summary()

predictions = tf.model.predict(x_one_hot)

for i,x in enumerate(predictions): # [] 하나 벗기는데 쓴듯?
    print(x)
    result_str = [idx2char[c] for c in np.argmax(x, axis=1)]
    print("Prediction str :", ''.join(result_str))

(1, 6, 5)
(1, 6, 5)
Epoch 1/50
1/1 [==============================] - 1s 894ms/step - loss: 1.6165 - accuracy: 0.1667
Epoch 2/50
1/1 [==============================] - 0s 4ms/step - loss: 1.5054 - accuracy: 0.3333
Epoch 3/50
1/1 [==============================] - 0s 4ms/step - loss: 1.4050 - accuracy: 0.3333
Epoch 4/50
1/1 [==============================] - 0s 4ms/step - loss: 1.3260 - accuracy: 0.3333
Epoch 5/50
1/1 [==============================] - 0s 5ms/step - loss: 1.2441 - accuracy: 0.3333
Epoch 6/50
1/1 [==============================] - 0s 5ms/step - loss: 1.1479 - accuracy: 0.3333
Epoch 7/50
1/1 [==============================] - 0s 5ms/step - loss: 1.0641 - accuracy: 0.5000
Epoch 8/50
1/1 [==============================] - 0s 4ms/step - loss: 0.9776 - accuracy: 0.6667
Epoch 9/50
1/1 [==============================] - 0s 4ms/step - loss: 0.8930 - accuracy: 0.6667
Epoch 10/50
1/1 [==============================] - 0s 4ms/step - loss: 0.8230 - accuracy: 0.6667
Epoch 11/50
1/1 [

In [10]:
# Lab 12 Character Sequence RNN
import tensorflow as tf
import numpy as np

sample = " if you want you"
idx2char = list(set(sample))  # index -> char
char2idx = {c: i for i, c in enumerate(idx2char)}  # char -> idex

# hyper parameters
dic_size = len(char2idx)  # RNN input size (one hot size)
hidden_size = len(char2idx)  # RNN output size
num_classes = len(char2idx)  # final output size (RNN or softmax, etc.)
batch_size = 1  # one sample data, one batch
sequence_length = len(sample) - 1  # number of lstm rollings (unit #)
learning_rate = 0.1

sample_idx = [char2idx[c] for c in sample]  # char to index
x_data = [sample_idx[:-1]]  # X data sample (0 ~ n-1) hello: hell
y_data = [sample_idx[1:]]   # Y label sample (1 ~ n) hello: ello

x_one_hot_eager = tf.one_hot(x_data, num_classes)  # one hot: 1 -> 0 1 0 0 0 0 0 0 0 0
x_one_hot_numpy = tf.keras.utils.to_categorical(x_data, num_classes)  # it'll generate numpy array, either way works
y_one_hot_eager = tf.one_hot(y_data, num_classes)

tf.model = tf.keras.Sequential();
tf.model.add(tf.keras.layers.
             LSTM(units=num_classes, input_shape=(sequence_length, x_one_hot_eager.shape[2]), return_sequences=True))
tf.model.add(tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(units=num_classes, activation='softmax')))
tf.model.summary()
tf.model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(lr=learning_rate),
                 metrics=['accuracy'])
tf.model.fit(x_one_hot_eager, y_one_hot_eager, epochs=50)


predictions = tf.model.predict(x_one_hot_eager)

for i, prediction in enumerate(predictions):
    # print char using argmax, dict
    result_str = [idx2char[c] for c in np.argmax(prediction, axis=1)]
    print("\tPrediction str: ", ''.join(result_str))

(1, 15, 10)
Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 15, 10)            840       
                                                                 
 time_distributed_6 (TimeDis  (None, 15, 10)           110       
 tributed)                                                       
                                                                 
Total params: 950
Trainable params: 950
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
1/1 [==============================] - 1s 1s/step - loss: 2.3327 - accuracy: 0.0000e+00
Epoch 2/50
1/1 [==============================] - 0s 7ms/step - loss: 2.1859 - accuracy: 0.3333
Epoch 3/50
1/1 [==============================] - 0s 8ms/step - loss: 2.0485 - accuracy: 0.2000
Epoch 4/50
1/1 [==============================] - 0s 7ms/step - loss: 1.8698 - accu

In [9]:
sample ="if you want you"
idx2char = list(set(sample))
print(idx2char)
char2idx = {c:i for t,c in enumerate(idx2char)}
print(char2idx)
print(len(char2idx))

['i', 't', 'f', 'u', 'y', 'w', 'a', ' ', 'o', 'n']
{'i': 0, 't': 0, 'f': 0, 'u': 0, 'y': 0, 'w': 0, 'a': 0, ' ': 0, 'o': 0, 'n': 0}
10


In [17]:
# Lab 12 Character Sequence RNN
import numpy as np
import tensorflow as tf

sample = " if you want you"
idx2char = list(set(sample))
char2idx = {c:i for i,c in enumerate(idx2char)}

dic_size = len(char2idx)
hidden_size = len(char2idx)
num_classes = len(char2idx)
batch_size = 1
sequence_size = len(sample) - 1
learning_rate = 0.1

sample_idx = [char2idx[c] for c in sample]
x_data = [sample_idx[:-1]]
y_data = [sample_idx[1:]]

x_one_hot = tf.keras.utils.to_categorical(x_data, num_classes)
y_one_hot = tf.keras.utils.to_categorical(y_data, num_classes)

tf.model = tf.keras.Sequential()
tf.model.add(tf.keras.layers.
             LSTM(units=num_classes, input_shape=(sequence_size, x_one_hot.shape[2]), return_sequences=True))
tf.model.add(tf.keras.layers.
             TimeDistributed(tf.keras.layers.
                             Dense(units=num_classes, activation='softmax')))
tf.model.summary()
tf.model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), metrics=['accuracy'])
tf.model.fit(x_one_hot, y_one_hot, epochs=50)

predictions = tf.model.predict(x_one_hot)

for i, x in enumerate(predictions):
    result_str = [idx2char[c] for c in np.argmax(x, axis=1)]
    print("Prediction str: ", ''.join(result_str))

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_5 (LSTM)               (None, 15, 10)            840       
                                                                 
 time_distributed_10 (TimeDi  (None, 15, 10)           110       
 stributed)                                                      
                                                                 
Total params: 950
Trainable params: 950
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
1/1 [==============================] - 1s 1s/step - loss: 2.2931 - accuracy: 0.2667
Epoch 2/50
1/1 [==============================] - 0s 6ms/step - loss: 2.1281 - accuracy: 0.2000
Epoch 3/50
1/1 [==============================] - 0s 7ms/step - loss: 2.0305 - accuracy: 0.2000
Epoch 4/50
1/1 [==============================] - 0s 6ms/step - loss: 1.8647 - accuracy: 0.3333
Ep

In [19]:
import tensorflow as tf
import numpy as np

sentence = ("if you want to build a ship, don't drum up people together to "
            "collect wood and don't assign them tasks and work, but rather "
            "teach them to long for the endless immensity of the sea.")

char_set = list(set(sentence))
char_dic = {w: i for i, w in enumerate(char_set)}

data_dim = len(char_set)
hidden_size = len(char_set)
num_classes = len(char_set)
sequence_length = 10  # Any arbitrary number / 문장을 나누는 구간 길이
learning_rate = 0.1

dataX = []
dataY = []
for i in range(0, len(sentence) - sequence_length):
    x_str = sentence[i:i + sequence_length]
    y_str = sentence[i + 1: i + sequence_length + 1]
    print(i, x_str, '->', y_str) ##학습과정 보여주기용(?)

    x = [char_dic[c] for c in x_str]  # x str to index  ## 각 글자를 index로 변경
    y = [char_dic[c] for c in y_str]  # y str to index

    dataX.append(x) ## for문 돌리면서 append로 리스트에 추가해서 dataX 완성
    dataY.append(y)

batch_size = len(dataX)

# One-hot encoding
X_one_hot = tf.one_hot(dataX, num_classes)
Y_one_hot = tf.one_hot(dataY, num_classes)

print(X_one_hot.shape)  # check out the shape (170, 10, 25)
print(Y_one_hot.shape)  # check out the shape


tf.model = tf.keras.Sequential();
tf.model.add(tf.keras.layers.
             LSTM(units=num_classes, input_shape=(sequence_length, X_one_hot.shape[2]), return_sequences=True))
tf.model.add(tf.keras.layers.LSTM(units=num_classes, return_sequences=True))
tf.model.add(tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(units=num_classes, activation='softmax')))
tf.model.summary()
tf.model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(lr=learning_rate),
                 metrics=['accuracy'])
tf.model.fit(X_one_hot, Y_one_hot, epochs=100)


results = tf.model.predict(X_one_hot)
for j, result in enumerate(results):
    index = np.argmax(result, axis=1)
    if j == 0:  # print all for the first result to make a sentence
        print(''.join([char_set[t] for t in index]), end='')
    else:
        print(char_set[index[-1]], end='')
        #첫번째 결과만 전부 쓰고, 그 이후의 결과는 마지막 글자만 쌓는다.
        #(결과가 한칸씩 밀려서 마지막에 새로운 글자가 나오기 때문에 그래야 문장을 만들수 있음)
        # end='' 는 print문을 연속으로 사용할때 줄 바꿈이 되지않고 바로 이어서 print 되게끔 해준다.

0 if you wan -> f you want
1 f you want ->  you want 
2  you want  -> you want t
3 you want t -> ou want to
4 ou want to -> u want to 
5 u want to  ->  want to b
6  want to b -> want to bu
7 want to bu -> ant to bui
8 ant to bui -> nt to buil
9 nt to buil -> t to build
10 t to build ->  to build 
11  to build  -> to build a
12 to build a -> o build a 
13 o build a  ->  build a s
14  build a s -> build a sh
15 build a sh -> uild a shi
16 uild a shi -> ild a ship
17 ild a ship -> ld a ship,
18 ld a ship, -> d a ship, 
19 d a ship,  ->  a ship, d
20  a ship, d -> a ship, do
21 a ship, do ->  ship, don
22  ship, don -> ship, don'
23 ship, don' -> hip, don't
24 hip, don't -> ip, don't 
25 ip, don't  -> p, don't d
26 p, don't d -> , don't dr
27 , don't dr ->  don't dru
28  don't dru -> don't drum
29 don't drum -> on't drum 
30 on't drum  -> n't drum u
31 n't drum u -> 't drum up
32 't drum up -> t drum up 
33 t drum up  ->  drum up p
34  drum up p -> drum up pe
35 drum up pe -> rum up peo
36

In [1]:
import tensorflow as tf
import numpy as np

sentence = ("if you want to build a ship, don't drum up people together to "
            "collect wood and don't assign them tasks and work, but rather "
            "teach them to long for the endless immensity of the sea.")
char_set = list(set(sentence))
char_dic = {w:i for i,w in enumerate(char_set)}

data_dim = len(char_set)
hidden_size = len(char_set)
num_classes = len(char_set)
sequence_length = 10
learning_rate = 0.1

dataX = []
dataY = []
for i in range(0, len(sentence) - sequence_length):
    x_str = sentence[i:i+sequence_length]
    y_str = sentence[i+1:i+sequence_length+1]
    print(i,x_str,'->',y_str)
    
    x = [char_dic[c] for c in x_str]
    y = [char_dic[c] for c in y_str]
    
    dataX.append(x)
    dataY.append(y)
    
batch_size = len(dataX)


X_one_hot = tf.one_hot(dataX, num_classes)
Y_one_hot = tf.one_hot(dataY, num_classes)

print(X_one_hot.shape)
print(Y_one_hot.shape)

tf.model = tf.keras.Sequential()
tf.model.add(tf.keras.layers.
            LSTM(units=num_classes, input_shape=(sequence_length, X_one_hot.shape[2]), return_sequences =True))
tf.model.add(tf.keras.layers.LSTM(units=num_classes, return_sequences=True))
tf.model.add(tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(units=num_classes, activation='softmax')))
tf.model.summary()
tf.model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate), metrics=['accuracy'])
tf.model.fit(X_one_hot, Y_one_hot, epochs=100)

results = tf.model.predict(X_one_hot)
for j, result in enumerate(results):
    index = np.argmax(result, axis=1)
    if j==0 :
        print(''.join([char_set[t] for t in index]), end='')
    else:
        print(char_set[index[-1]], end='')
        

0 if you wan -> f you want
1 f you want ->  you want 
2  you want  -> you want t
3 you want t -> ou want to
4 ou want to -> u want to 
5 u want to  ->  want to b
6  want to b -> want to bu
7 want to bu -> ant to bui
8 ant to bui -> nt to buil
9 nt to buil -> t to build
10 t to build ->  to build 
11  to build  -> to build a
12 to build a -> o build a 
13 o build a  ->  build a s
14  build a s -> build a sh
15 build a sh -> uild a shi
16 uild a shi -> ild a ship
17 ild a ship -> ld a ship,
18 ld a ship, -> d a ship, 
19 d a ship,  ->  a ship, d
20  a ship, d -> a ship, do
21 a ship, do ->  ship, don
22  ship, don -> ship, don'
23 ship, don' -> hip, don't
24 hip, don't -> ip, don't 
25 ip, don't  -> p, don't d
26 p, don't d -> , don't dr
27 , don't dr ->  don't dru
28  don't dru -> don't drum
29 don't drum -> on't drum 
30 on't drum  -> n't drum u
31 n't drum u -> 't drum up
32 't drum up -> t drum up 
33 t drum up  ->  drum up p
34  drum up p -> drum up pe
35 drum up pe -> rum up peo
36

6/6 [==============================] - 0s 6ms/step - loss: 0.3029 - accuracy: 0.8806
Epoch 27/100
6/6 [==============================] - 0s 6ms/step - loss: 0.3038 - accuracy: 0.8812
Epoch 28/100
6/6 [==============================] - 0s 6ms/step - loss: 0.2916 - accuracy: 0.8894
Epoch 29/100
6/6 [==============================] - 0s 6ms/step - loss: 0.2893 - accuracy: 0.8847
Epoch 30/100
6/6 [==============================] - 0s 6ms/step - loss: 0.2826 - accuracy: 0.8853
Epoch 31/100
6/6 [==============================] - 0s 6ms/step - loss: 0.2841 - accuracy: 0.8853
Epoch 32/100
6/6 [==============================] - 0s 6ms/step - loss: 0.2862 - accuracy: 0.8847
Epoch 33/100
6/6 [==============================] - 0s 6ms/step - loss: 0.2833 - accuracy: 0.8871
Epoch 34/100
6/6 [==============================] - 0s 6ms/step - loss: 0.2897 - accuracy: 0.8841
Epoch 35/100
6/6 [==============================] - 0s 6ms/step - loss: 0.2856 - accuracy: 0.8853
Epoch 36/100
6/6 [===============